## Training models
Requires a zip file containing preprocessed images into *AN2DL_Challenge2-TheBigBatchTheory/data/processed*.

### Preamble
Drive connection, setup for fast loading, Github repo connection

In [1]:
from google.colab import drive

# 1. Mount Drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import shutil
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import argparse
import zipfile

# =============================================================================
# --- 1. CONFIGURAZIONE GLOBALE ---
# =============================================================================
TILE_SIZES = {
    'context': 768,
    'detail': 256
}
MIN_ROI_AREA = 100

# =============================================================================
# --- 2. FUNZIONI DI UTILITÀ E I/O (Invariate) ---
# =============================================================================
def load_image_cv2(path: Path):
    return cv2.imdecode(np.fromfile(str(path), dtype=np.uint8), cv2.IMREAD_COLOR)

def load_mask_cv2(path: Path):
    return cv2.imdecode(np.fromfile(str(path), dtype=np.uint8), cv2.IMREAD_GRAYSCALE)


# =============================================================================
# --- 3. CONTROLLO QUALITÀ SULL'INTERA SLIDE (Le tue funzioni) ---
# =============================================================================
def contains_slime(img_bgr, threshold=50):
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    lower_green = np.array([35, 50, 50])
    upper_green = np.array([90, 255, 255])
    mask_slime = cv2.inRange(hsv, lower_green, upper_green)
    return cv2.countNonZero(mask_slime) > threshold

def analyze_image_memory(img_bgr):
    if img_bgr is None: return "FAIL"
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    mask_foreground = (hsv[:,:,1] > 15) & (hsv[:,:,2] < 250)
    foreground_pixels = np.count_nonzero(mask_foreground)
    if foreground_pixels < 100: return "SAFE"
    h_foreground = hsv[:,:,0][mask_foreground]
    count_tissue = np.count_nonzero((h_foreground >= 125) & (h_foreground <= 175))
    count_ink = np.count_nonzero((h_foreground >= 80) & (h_foreground < 125))
    count_shrek_skin = np.count_nonzero((h_foreground >= 20) & (h_foreground < 80))
    count_shrek_clothes = np.count_nonzero((h_foreground >= 10) & (h_foreground < 20))
    count_shrek_total = count_shrek_skin + count_shrek_clothes
    if count_tissue == 0: return "SHREK" if count_shrek_total > 0 else "SAFE"
    ratio_tissue, ratio_shrek = count_tissue/foreground_pixels, count_shrek_total/foreground_pixels
    shrek_dominance = count_shrek_total / count_tissue
    if (count_ink/foreground_pixels) > ratio_shrek and (count_ink/foreground_pixels) > 0.1: return "SAFE"
    if ratio_shrek > 0.4 and shrek_dominance > 4.0: return "SHREK"
    if ratio_tissue > 0.05: return "SAFE"
    if ratio_shrek > 0.3: return "SHREK"
    return "SAFE"

# =============================================================================
# --- 4. MOTORE DI TILING (Invariato dalla versione multi-modale) ---
# =============================================================================
def process_single_slide_multimodal(img_bgr, roi_mask, label, img_path, output_dirs, is_test_set=False):
    # ... (questa funzione rimane identica, dato che il QC è ora nel loop principale)
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    _, tissue_mask = cv2.threshold(img_gray, 220, 255, cv2.THRESH_BINARY_INV)
    img_masked = img_bgr.copy()
    img_masked[tissue_mask == 0] = 255
    contours, _ = cv2.findContours(roi_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    tiles_data = []
    base_name = img_path.stem
    if not contours: return None
    for i, contour in enumerate(contours):
        if cv2.contourArea(contour) < MIN_ROI_AREA: continue
        M = cv2.moments(contour)
        if M["m00"] == 0: continue
        center_x, center_y = int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])
        tile_name_base = f"{base_name}_roi{i}"
        for scale_name, tile_size in TILE_SIZES.items():
            half_tile = tile_size // 2
            y_start, y_end = center_y - half_tile, center_y + half_tile
            x_start, x_end = center_x - half_tile, center_x + half_tile
            img_crop = img_masked[max(0, y_start):min(img_masked.shape[0], y_end), max(0, x_start):min(img_masked.shape[1], x_end)]
            mask_crop = roi_mask[max(0, y_start):min(roi_mask.shape[0], y_end), max(0, x_start):min(roi_mask.shape[1], x_end)]
            pad_top, pad_bottom = max(0, -y_start), max(0, y_end - img_masked.shape[0])
            pad_left, pad_right = max(0, -x_start), max(0, x_end - img_masked.shape[1])
            if any([pad_top, pad_bottom, pad_left, pad_right]):
                img_crop = cv2.copyMakeBorder(img_crop, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=[255, 255, 255])
                mask_crop = cv2.copyMakeBorder(mask_crop, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=[0])
            tile_name = f"{tile_name_base}.png"
            cv2.imwrite(str(output_dirs[f'images_{scale_name}'] / tile_name), img_crop)
            cv2.imwrite(str(output_dirs[f'masks_{scale_name}'] / tile_name), mask_crop)
        row = {'sample_index': f"{tile_name_base}.png", 'original_sample': img_path.name}
        if not is_test_set: row['label'] = label
        tiles_data.append(row)
    return tiles_data

# =============================================================================
# --- 5. WORKFLOW PRINCIPALE (CON QC REINTEGRATO CORRETTAMENTE) ---
# =============================================================================
def run_pipeline(train_dir: Path, labels_csv: Path, output_dir: Path, test_dir: Path = None):
    if output_dir.exists(): shutil.rmtree(output_dir)

    # --- Process TRAINING SET ---
    output_dirs_train = { f"images_{name}": output_dir / f"train/images_{name}" for name in TILE_SIZES.keys() }
    output_dirs_train.update({ f"masks_{name}": output_dir / f"train/masks_{name}" for name in TILE_SIZES.keys() })
    for d in output_dirs_train.values(): d.mkdir(parents=True, exist_ok=True)

    print("\n>>> FASE 1: Processing TRAINING SET (Multi-Modale con QC)")
    if not (train_dir.exists() and labels_csv.exists()):
        print(f"❌ ERROR: Training directory o labels CSV non trovati. Interruzione.")
        return

    labels_df = pd.read_csv(labels_csv)
    all_train_tiles = []

    for _, row in tqdm(labels_df.iterrows(), total=len(labels_df), desc="Train Slides"):
        fname, label = row['sample_index'], row['label']
        img_paths = list(train_dir.glob(f"**/{fname}"))
        if not img_paths: continue
        img_path = img_paths[0]

        mask_path = img_path.parent / fname.replace("img_", "mask_")
        if not mask_path.exists(): mask_path = mask_path.with_suffix('.png')
        if not mask_path.exists(): continue

        img_bgr = load_image_cv2(img_path)
        roi_mask = load_mask_cv2(mask_path)
        if img_bgr is None or roi_mask is None: continue

        # --- CONTROLLO QUALITÀ SULL'INTERA SLIDE ---
        if contains_slime(img_bgr) or analyze_image_memory(img_bgr) == "SHREK":
            # Stampa un messaggio opzionale per il debug
            # print(f"\nINFO: Skipping {fname} (QC Failed)")
            continue
        # --- FINE CONTROLLO QUALITÀ ---

        tiles = process_single_slide_multimodal(img_bgr, roi_mask, label, img_path, output_dirs_train, is_test_set=False)
        if tiles: all_train_tiles.extend(tiles)

    if all_train_tiles:
        train_df = pd.DataFrame(all_train_tiles)
        out_train_csv = output_dir / "train/train_patches.csv"
        train_df.to_csv(out_train_csv, index=False)
        print(f"✅ Training Set Completo. Salvati {len(all_train_tiles)} multimodal tile sets.")

    # --- Process TEST SET (logica completa inclusa) ---
    if test_dir and test_dir.exists():
        output_dirs_test = { f"images_{name}": output_dir / f"test/images_{name}" for name in TILE_SIZES.keys() }
        output_dirs_test.update({ f"masks_{name}": output_dir / f"test/masks_{name}" for name in TILE_SIZES.keys() })
        for d in output_dirs_test.values(): d.mkdir(parents=True, exist_ok=True)

        print("\n>>> FASE 2: Processing TEST SET (Multi-Modale)")
        all_test_tiles = []
        test_images = sorted([p for p in test_dir.rglob("img_*.*") if "mask" not in p.name])

        for img_path in tqdm(test_images, desc="Test Slides"):
            id_part = img_path.stem.replace("img_", "")
            mask_path = img_path.parent / f"mask_{id_part}{img_path.suffix}"
            if not mask_path.exists(): mask_path = mask_path.with_suffix('.png')
            if not mask_path.exists(): continue

            img_bgr = load_image_cv2(img_path)
            roi_mask = load_mask_cv2(mask_path)
            if img_bgr is None or roi_mask is None: continue

            # NOTA: il QC non viene applicato al test set
            tiles = process_single_slide_multimodal(img_bgr, roi_mask, None, img_path, output_dirs_test, is_test_set=True)
            if tiles: all_test_tiles.extend(tiles)

        if all_test_tiles:
            test_df = pd.DataFrame(all_test_tiles)
            out_test_csv = output_dir / "test/test_patches.csv"
            test_df.to_csv(out_test_csv, index=False)
            print(f"✅ Test Set Completo. Salvati {len(all_test_tiles)} multimodal tile sets.")
    else:
        print("\nℹ️ Nessuna directory di test fornita o trovata, salto il processing del test set.")

def zip_directory(folder_path: Path, zip_path: Path):
  print(f"\n>>> Zipping output directory to '{zip_path}'...")
  files_to_zip = [entry for entry in folder_path.rglob('*') if entry.is_file()]
  with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
      for entry in tqdm(files_to_zip, desc="Zipping files"):
          zipf.write(entry, entry.relative_to(folder_path))
  print(f"✅ Zipping complete.")
# =============================================================================
# --- 5. ENTRY POINT (Invariato) ---
# =============================================================================
if __name__ == "__main__":
    BASE_PATH = Path('/content/drive/MyDrive/AN2DL_Challenge2-TheBigBatchTheory/data')
    TRAIN_DATA_DIR = BASE_PATH / 'dataset/train_data'
    TEST_DATA_DIR = BASE_PATH / 'dataset/test_data'
    LABELS_CSV_PATH = BASE_PATH / 'dataset/train_labels.csv'
    OUTPUT_PREPROCESSED_DIR = BASE_PATH / 'preprocessed/preprocessed_MaskTile'
    OUTPUT_ZIP_PATH = BASE_PATH / 'preprocessed/preprocessed_MaskTile.zip'

    run_pipeline(
        train_dir=TRAIN_DATA_DIR,
        labels_csv=LABELS_CSV_PATH,
        output_dir=OUTPUT_PREPROCESSED_DIR,
        test_dir=TEST_DATA_DIR
    )
    zip_directory(OUTPUT_PREPROCESSED_DIR, OUTPUT_ZIP_PATH)




>>> FASE 1: Processing TRAINING SET (Multi-Modale con QC)


Train Slides: 100%|██████████| 691/691 [08:04<00:00,  1.43it/s]


✅ Training Set Completo. Salvati 4573 multimodal tile sets.

>>> FASE 2: Processing TEST SET (Multi-Modale)


Test Slides: 100%|██████████| 477/477 [05:20<00:00,  1.49it/s]


✅ Test Set Completo. Salvati 3596 multimodal tile sets.

>>> Zipping output directory to '/content/drive/MyDrive/AN2DL_Challenge2-TheBigBatchTheory/data/preprocessed/preprocessed_MaskTile.zip'...


In [2]:
import os, sys, random, subprocess, shutil
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import time
!pip install -q comet_ml torchsummary # Uncomment if running in a fresh cell
from comet_ml import start
from comet_ml.integration.pytorch import log_model
from torch import nn
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import cv2
from PIL import Image
from tqdm.notebook import tqdm

logs_dir = "tensorboard"
# !pkill -f tensorboard # Uncomment if needed
# %load_ext tensorboard # Uncomment if needed
# !mkdir -p models      # Uncomment if needed

# 2. Setup Seed e Device
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 3. Setup Github Repository
REPO = "an2dl-challenges-25-26"
TARGET_FOLDER = "challenge2"

# Clone repo (Assuming you are in Colab root /content/)
if os.path.exists(REPO):
    print(f"Folder '{REPO}' already exists. Deleting it for a fresh clone...")
    shutil.rmtree(REPO)

# Now clone it fresh
subprocess.run(["git", "clone", "-b", "francesco2", "--single-branch", f"https://github.com/asarraa/{REPO}.git"])

# 3. Navigate INTO the subfolder
project_path = os.path.abspath(os.path.join(os.getcwd(), REPO, TARGET_FOLDER))

# Add to Python Path
if project_path not in sys.path:
    sys.path.append(project_path)

# Change Directory
os.chdir(project_path)

print(f"Setup Github complete.")
print(f"Current Working Directory: {os.getcwd()}")

# --------------------------------------------------------------------------
# 4. SETUP DATI VELOCI (DRIVE -> LOCALE SSD)
# --------------------------------------------------------------------------
DATA_VARIANT = 'preprocessed_MaskTile'
DRIVE_ROOT = Path('/content/drive/MyDrive/AN2DL_Challenge2-TheBigBatchTheory/')
DRIVE_DATA_ROOT = DRIVE_ROOT / 'data/preprocessed'

zip_path = DRIVE_DATA_ROOT / f"{DATA_VARIANT}.zip"
non_zip_path = DRIVE_DATA_ROOT / f"{DATA_VARIANT}"

LOCAL_DATA_ROOT = Path('/content/local_data')
LOCAL_DATA_ROOT.mkdir(parents=True, exist_ok=True) # Ensure local root exists

# Logic: Check Zip -> Fallback to Folder -> Error
ZIP = True
if zip_path.exists() and ZIP:
    print(f"Trovato zip in: {zip_path}")
    print("Copia ed estrazione in corso su SSD locale (richiede ~30-60 sec)...")

    temp_zip = Path("/content/temp_dataset.zip")
    shutil.copy(zip_path, temp_zip)
    shutil.unpack_archive(temp_zip, LOCAL_DATA_ROOT)
    os.remove(temp_zip)
    print("Estrazione completata!")

elif non_zip_path.exists():
    print(f"Zip NON trovato. Trovata cartella in: {non_zip_path}")
    print("Copia ricorsiva in corso su SSD locale (potrebbe essere più lento del zip)...")
    # Destination path for the folder copy
    destination_dir = LOCAL_DATA_ROOT / DATA_VARIANT
    # Remove destination if it already exists to avoid errors
    if destination_dir.exists():
        shutil.rmtree(destination_dir)
    # Use copytree for directories
    shutil.copytree(non_zip_path, destination_dir)
    print("Copia cartella completata!")

else:
    raise FileNotFoundError(
        f"ERRORE CRITICO: Non trovo né {DATA_VARIANT}.zip né la cartella {DATA_VARIANT} "
        f"in: {DRIVE_DATA_ROOT}"
    )

# --------------------------------------------------------------------------
# 5. DEFINIZIONE PERCORSI
# --------------------------------------------------------------------------
# Check structure: Did the zip unpack into a subfolder or directly?
if (LOCAL_DATA_ROOT / DATA_VARIANT).exists():
    BASE_PATH = LOCAL_DATA_ROOT / DATA_VARIANT
else:
    BASE_PATH = LOCAL_DATA_ROOT

TRAIN_IMG_DIR = BASE_PATH / 'train/images'
TEST_IMG_DIR = BASE_PATH / 'test/images'
TRAIN_CSV_PATH = BASE_PATH / 'train/train_patches.csv'

print(f'Using device: {device}')
print(f'DATASET PRONTO IN LOCALE: {BASE_PATH}')




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00
Setup Github complete.
Current Working Directory: /content/an2dl-challenges-25-26/challenge2
Trovato zip in: /content/drive/MyDrive/AN2DL_Challenge2-TheBigBatchTheory/data/preprocessed/preprocessed_MaskTile.zip
Copia ed estrazione in corso su SSD locale (richiede ~30-60 sec)...
Estrazione completata!
Using device: cuda
DATASET PRONTO IN LOCALE: /content/local_data


In [3]:
DATA_VARIANT = 'preprocessed_MaskTile'


LOCAL_DATA_ROOT = Path('/content/local_data')
BASE_PATH = LOCAL_DATA_ROOT


### Dataloaders

In [4]:
import lazy_loaders
import multiscale_pipeline

BATCH_SIZE = 16
ADD_MASK_CHANNEL = False
#
#train_loader, val_loader, input_shape, class_weights = multiscale_pipeline.get_multiscale_loaders(batch_size=BATCH_SIZE, base_path=BASE_PATH)
#test_loader = multiscale_pipeline.get_multiscale_test_loader(batch_size=BATCH_SIZE, base_path=BASE_PATH)

### Training

In [5]:
from launch_training import start_training
from torchvision import transforms
# Ensure 'models' is importable. If imports fail, restart kernel and run Cell 1 only.

MODEL_NAME = "AttentionMIL"
TRAINING_PARAMS = {
    'epochs': 1000,
    'patience':30,
    "l2_lambda": 1e-2,
    "learning_rate": 1e-04,
    "verbose": 1
}

MIL_DEFAULT = {"num_classes": 4, "pretrained": True, "freeze_backbone": True}



# Note: Ensure you imported comet_ml at the top if you want logging
trained_model, history, exp_id = start_training(
    model_name=MODEL_NAME,
    training_params=TRAINING_PARAMS,
    model_params = MIL_DEFAULT,
    local_data_path = BASE_PATH,
    batch_size = BATCH_SIZE,
    data_path = BASE_PATH
)

Using GPU: Tesla T4
--- Starting AttentionMIL on cuda ---


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


Starting AttentionMIL model training...
Training Configuration:
 epochs: 1000
learning_rate: 0.0001
patience: 30
l1_lambda: 0
l2_lambda: 0.01
verbose: 1
criterion_name: CrossEntropyLoss
optimizer_name: adamw
Model Configuration:
 num_classes: 4
backbone_name: vit_small_patch16_224
pretrained: True
freeze_backbone: True


COMET INFO: Experiment is live on comet.com https://www.comet.com/asarraa/test/762e7514b13145cf9dce60913268a036

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

...

Epoch  34/1000 | Train: Loss=0.9506, F1=0.6837 | Val: Loss=1.4357, F1=0.3232
   - No improvement. Patience: 27/30


Epoch  35/1000 | Train: Loss=0.9419, F1=0.6614 | Val: Loss=1.4364, F1=0.3249
   - No improvement. Patience: 28/30


Epoch  36/1000 | Train: Loss=0.9318, F1=0.6778 | Val: Loss=1.4434, F1=0.3368
   - No improvement. Patience: 29/30


Epoch  37/1000 | Train: Loss=0.9169, F1=0.7103 | Val: Loss=1.4673, F1=0.2931
   - No improvement. Patience: 30/30

🛑 Early stopping triggered after 37 epochs.

--- Training Finished ---
Restoring best model weights from epoch 7 with val_f1 of 0.3601
Model saved to: /content/local_data/experiments/models/AttentionMIL_20251216_210310.pt
Registry updated: ID AttentionMIL_20251216_210310


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : AttentionMIL_20251216_210310
COMET INFO:     url                   : https://www.comet.com/asarraa/test/762e7514b13145cf9dce60913268a036
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     best_val_f1 [5] : (0.258416336741525, 0.3601266422131213)
COMET INFO:     train_f1 [37]   : (0.2789116870054049, 0.7103098496364149)
COMET INFO:     train_loss [37] : (0.9169371025315647, 1.425970278937241)
COMET INFO:     val_f1 [37]     : (0.258416336741525, 0.3601266422131213)
COMET INFO:     val_loss [37]   : (1.340187385550931, 1.4673121698901184)
COMET INFO:   Others:
COMET INFO:     Name                : AttentionMIL_20251216_210310
COMET INFO:     loc

In [7]:
MODEL_NAME = "AttentionMIL"
TRAINING_PARAMS = {
    'epochs': 1000,
    'patience':30,
    "l2_lambda": 1e-2,
    "learning_rate": 1e-05,
    "verbose": 1
}

MIL_DEFAULT = {"num_classes": 4, "pretrained": True, "freeze_backbone": False}

trained_model, history, exp_id = start_training(
    pretrained_model_path = '/content/local_data/fit_models/AttentionMIL_20251216_210310_best_model.pth',
    model_name=MODEL_NAME,
    training_params=TRAINING_PARAMS,
    model_params = MIL_DEFAULT,
    local_data_path = BASE_PATH,
    batch_size = BATCH_SIZE,
    data_path = BASE_PATH
)

Using GPU: Tesla T4
--- Starting AttentionMIL on cuda ---


COMET INFO: An experiment with the same configuration options is already running and will be reused.


Starting AttentionMIL model training...
Training Configuration:
 epochs: 1000
learning_rate: 1e-05
patience: 30
l1_lambda: 0
l2_lambda: 0.01
verbose: 1
criterion_name: CrossEntropyLoss
optimizer_name: adamw
Model Configuration:
 num_classes: 4
backbone_name: vit_small_patch16_224
pretrained: True
freeze_backbone: False
Loading pretrained model from /content/local_data/fit_models/AttentionMIL_20251216_210310_best_model.pth...
Pretrained model loaded successfully.


Epoch   1/1000 | Train: Loss=1.2612, F1=0.4018 | Val: Loss=1.3459, F1=0.3437

✅ Improvement! val_f1 changed from -inf to 0.3437.
   - Saving best model weights to '/content/local_data/fit_models/AttentionMIL_20251216_212112_best_model.pth'


Epoch   2/1000 | Train: Loss=1.2062, F1=0.4509 | Val: Loss=1.3521, F1=0.3287
   - No improvement. Patience: 1/30


...

Epoch  78/1000 | Train: Loss=0.3678, F1=1.0000 | Val: Loss=1.6860, F1=0.3771
   - No improvement. Patience: 27/30


Epoch  79/1000 | Train: Loss=0.3703, F1=1.0000 | Val: Loss=1.6422, F1=0.3696
   - No improvement. Patience: 28/30


Epoch  80/1000 | Train: Loss=0.3684, F1=1.0000 | Val: Loss=1.6738, F1=0.3892
   - No improvement. Patience: 29/30


Epoch  81/1000 | Train: Loss=0.3689, F1=1.0000 | Val: Loss=1.7450, F1=0.3437
   - No improvement. Patience: 30/30

🛑 Early stopping triggered after 81 epochs.

--- Training Finished ---
Restoring best model weights from epoch 51 with val_f1 of 0.4011
Model saved to: /content/local_data/experiments/models/AttentionMIL_20251216_212112.pt
Registry updated: ID AttentionMIL_20251216_212112


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : AttentionMIL_20251216_212112
COMET INFO:     url                   : https://www.comet.com/asarraa/test/56af7d79438f43939a4f6fbd6706c905
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     best_val_f1 [6] : (0.3436692506459948, 0.40108315573155956)
COMET INFO:     train_f1 [81]   : (0.401810716789928, 1.0)
COMET INFO:     train_loss [81] : (0.36775538119776496, 1.2612053936925427)
COMET INFO:     val_f1 [81]     : (0.31599548929119436, 0.40108315573155956)
COMET INFO:     val_loss [81]   : (1.3458522874065952, 1.8052192223377717)
COMET INFO:   Others:
COMET INFO:     Name                : AttentionMIL_20251216_212112
COMET INFO:     local_exper

In [ ]:
MODEL_NAME = "AttentionMIL"
TRAINING_PARAMS = {
    'epochs': 1000,
    'patience':10,
    "l2_lambda": 1e-2,
    "learning_rate": 1e-06,
    "verbose": 1
}

MIL_DEFAULT = {"num_classes": 4, "pretrained": True, "freeze_backbone": False}

trained_model, history, exp_id = start_training(
    pretrained_model_path = '/content/local_data/fit_models/AttentionMIL_20251216_174926_best_model.pth',
    model_name=MODEL_NAME,
    training_params=TRAINING_PARAMS,
    model_params = MIL_DEFAULT,
    local_data_path = BASE_PATH,
    batch_size = BATCH_SIZE,
    data_path = BASE_PATH
)

Using GPU: Tesla T4
--- Starting AttentionMIL on cuda ---


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


Starting AttentionMIL model training...
Training Configuration:
 epochs: 1000
learning_rate: 1e-06
patience: 10
l1_lambda: 0
l2_lambda: 0.01
verbose: 1
criterion_name: CrossEntropyLoss
optimizer_name: adamw
Model Configuration:
 num_classes: 4
backbone_name: vit_small_patch16_224
pretrained: True
freeze_backbone: False


COMET INFO: Experiment is live on comet.com https://www.comet.com/asarraa/test/2e19ec1b55734495a4e5a8d802703d6e



Loading pretrained model from /content/local_data/fit_models/AttentionMIL_20251216_174926_best_model.pth...
Pretrained model loaded successfully.


...

Epoch  25/1000 | Train: Loss=1.0511, F1=0.6229 | Val: Loss=1.3500, F1=0.3724
   - No improvement. Patience: 7/10


Epoch  26/1000 | Train: Loss=1.0446, F1=0.6167 | Val: Loss=1.3526, F1=0.3576
   - No improvement. Patience: 8/10


Epoch  27/1000 | Train: Loss=1.0357, F1=0.6291 | Val: Loss=1.3552, F1=0.3576
   - No improvement. Patience: 9/10


Epoch  28/1000 | Train: Loss=1.0384, F1=0.6398 | Val: Loss=1.3570, F1=0.3708
   - No improvement. Patience: 10/10

🛑 Early stopping triggered after 28 epochs.

--- Training Finished ---
Restoring best model weights from epoch 18 with val_f1 of 0.3906
Model saved to: /content/local_data/experiments/models/AttentionMIL_20251216_181934.pt
Registry updated: ID AttentionMIL_20251216_181934


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : AttentionMIL_20251216_181934
COMET INFO:     url                   : https://www.comet.com/asarraa/test/2e19ec1b55734495a4e5a8d802703d6e
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     best_val_f1 [5] : (0.37962314127450775, 0.3906378138681229)
COMET INFO:     train_f1 [28]   : (0.43618726508781513, 0.6397776458322688)
COMET INFO:     train_loss [28] : (1.035734374975336, 1.2461730293158828)
COMET INFO:     val_f1 [28]     : (0.34711359372032735, 0.3906378138681229)
COMET INFO:     val_loss [28]   : (1.3429723621433616, 1.356974783106747)
COMET INFO:   Others:
COMET INFO:     Name                : AttentionMIL_20251216_181934
COMET INFO:   

In [ ]:
MODEL_NAME = "AttentionMIL"
TRAINING_PARAMS = {
    'epochs': 1000,
    'patience':10,
    "l2_lambda": 0.05,
    "learning_rate": 5e-06,
    "verbose": 1
}

MIL_DEFAULT = {"num_classes": 4, "pretrained": True, "freeze_backbone": False}

trained_model, history, exp_id = start_training(
    pretrained_model_path = '/content/local_data/fit_models/AttentionMIL_20251216_174926_best_model.pth',
    model_name=MODEL_NAME,
    training_params=TRAINING_PARAMS,
    model_params = MIL_DEFAULT,
    local_data_path = BASE_PATH,
    batch_size = BATCH_SIZE,
    data_path = BASE_PATH
)

Using GPU: Tesla T4
--- Starting AttentionMIL on cuda ---


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


Starting AttentionMIL model training...
Training Configuration:
 epochs: 1000
learning_rate: 5e-06
patience: 10
l1_lambda: 0
l2_lambda: 0.05
verbose: 1
criterion_name: CrossEntropyLoss
optimizer_name: adamw
Model Configuration:
 num_classes: 4
backbone_name: vit_small_patch16_224
pretrained: True
freeze_backbone: False


COMET INFO: Experiment is live on comet.com https://www.comet.com/asarraa/test/8cc557875973480db1fec8e8f8a78e26



Loading pretrained model from /content/local_data/fit_models/AttentionMIL_20251216_174926_best_model.pth...
Pretrained model loaded successfully.


...

Epoch  15/1000 | Train: Loss=0.6810, F1=0.8767 | Val: Loss=1.4977, F1=0.3612
   - No improvement. Patience: 7/10


Epoch  16/1000 | Train: Loss=0.6609, F1=0.8988 | Val: Loss=1.5479, F1=0.3152
   - No improvement. Patience: 8/10


Epoch  17/1000 | Train: Loss=0.6105, F1=0.9243 | Val: Loss=1.5452, F1=0.3635
   - No improvement. Patience: 9/10


Epoch  18/1000 | Train: Loss=0.5651, F1=0.9353 | Val: Loss=1.5921, F1=0.3600
   - No improvement. Patience: 10/10

🛑 Early stopping triggered after 18 epochs.

--- Training Finished ---
Restoring best model weights from epoch 8 with val_f1 of 0.3942
Model saved to: /content/local_data/experiments/models/AttentionMIL_20251216_184103.pt
Registry updated: ID AttentionMIL_20251216_184103


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : AttentionMIL_20251216_184103
COMET INFO:     url                   : https://www.comet.com/asarraa/test/8cc557875973480db1fec8e8f8a78e26
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     best_val_f1 [4] : (0.3626379115415971, 0.3941631298610704)
COMET INFO:     train_f1 [18]   : (0.4540904686388393, 0.9353160965189341)
COMET INFO:     train_loss [18] : (0.56509212892631, 1.253370460765115)
COMET INFO:     val_f1 [18]     : (0.3076946515860543, 0.3941631298610704)
COMET INFO:     val_loss [18]   : (1.3398393351807554, 1.5921241055186997)
COMET INFO:   Others:
COMET INFO:     Name                : AttentionMIL_20251216_184103
COMET INFO:     lo


### Inference on test set

In [8]:
from inference import make_mil_inference

# 1. Definisci i parametri
MODEL_PATH = Path('/content/local_data/fit_models/AttentionMIL_20251216_212112_best_model.pth')
BATCH_SIZE = 8 # Usa un batch size basso per l'inferenza MIL
EXPERIMENT_ID = "mil_inference_vit_01"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 2. Chiama la funzione principale di inferenza
make_mil_inference(
    model_path=MODEL_PATH,
    base_path=BASE_PATH,
    device=DEVICE,
    batch_size=BATCH_SIZE,
    experiment_id=EXPERIMENT_ID,
    backbone_name='vit_small_patch16_224'
)


[INFO] Starting MIL inference for experiment 'mil_inference_vit_01'...

--- Creating MIL Test DataLoader ---
[INFO] Loading AttentionMIL model with 4 classes and 'vit_small_patch16_224' backbone.
--- ATTENZIONE: Backbone è freezato. Solo la testa di classificazione verrà addestrata. ---
[INFO] Loading weights from: '/content/local_data/fit_models/AttentionMIL_20251216_212112_best_model.pth'
[INFO] Model loaded successfully.


Predicting on slides: 100%|██████████| 60/60 [00:19<00:00,  3.09it/s]


✅ Inference complete. Submission file saved to 'mil_inference_vit_01_submission.csv'
